Treinando uma rede neural usando pytorch
========================================



## Introdução



O notebook anterior foi o *season finale* da construção da rede neural usando Python puro. Este notebook é o epílogo.

Aqui veremos como podemos construir e treinar uma rede neural usando `pytorch`, um módulo especializado em redes neurais artificiais.

Motivos para usarmos o `pytorch`:

1.  A `MLP` que criamos em Python puro não é otimizada. Tente treinar uma rede com uns 100 neurônios em uma camada e verá como ela é bem lenta! `pytorch` é escrito por diversos programadores que além de primar pelas contas estarem corretas, eles fazem de tudo para que elas ocorram de forma eficiente.

2.  `pytorch` já tem praticamente tudo que precisamos implementado! Quer calcular o gradiente local da função arco tangente? Ele já tem! Quer usar a função de ativação ReLU? Ele já tem!

3.  `pytorch` tem suporte a treinar uma rede neural usando processamento gráfico (GPU). Isso acelera muito o treino de redes neurais artificiais complexas. Temos GPUs para usarmos no HPC da Ilum!

4.  `pytorch` é usado tanto na academia quanto no mundo corporativo. Junto com `tensorflow` são os dois módulos estado da arte para o projeto e treino de redes neurais.



## Objetivo



Treinar uma rede neural artificial tipo Multilayer Perceptron usando `pytorch`.



## Importações



In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

## Código e discussão



### Dados



Vamos treinar nossa rede neural com nosso velho amigo, o dataset de diamantes! Como todo processo de treino de aprendizado de máquina, precisamos reservar um conjunto de dados para treino e outro para teste.



In [2]:
TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 61455
DATASET_NAME = "diamonds"
FEATURES = ["carat", "depth", "table", "x", "y", "z"]
TARGET = ["price"]

df = sns.load_dataset(DATASET_NAME)

indices = df.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino = df.loc[indices_treino]
df_teste = df.loc[indices_teste]

X_treino = df_treino.reindex(FEATURES, axis=1)
y_treino = df_treino.reindex(TARGET, axis=1)
X_teste = df_teste.reindex(FEATURES, axis=1)
y_teste = df_teste.reindex(TARGET, axis=1)

Redes neurais costumam se dar bem quando os dados estão entre $[0, 1]$ ou entre $[-1, 1]$. Redes neurais costumam não se dar bem quando os dados estão em escalas muito diferentes. Por conta disso, normalizar os dados antes de prosseguir é recomendado! Vamos usar o `MinMaxScaler` do `scikit-learn` que aprendemos semestre passado.



In [3]:
normalizador_x = MinMaxScaler()
normalizador_y = MinMaxScaler()

normalizador_x.fit(X_treino)
normalizador_y.fit(y_treino)

X_treino = normalizador_x.transform(X_treino)
y_treino = normalizador_y.transform(y_treino)
X_teste = normalizador_x.transform(X_teste)
y_teste = normalizador_y.transform(y_teste)

### Tensores



Na nossa rede neural feita em Python puro, a base de tudo era a classe `Valor`. Para o `pytorch`, a base de tudo são os tensores! Tensores são objetos que armazenam dados de forma similar aos arrays de `numpy`, porém registram todas as operações realizadas para o cálculo do gradiente local (assim como nós fizemos com a classe `Valor`).



In [4]:
X_treino = torch.tensor(X_treino, dtype=torch.float32)
y_treino = torch.tensor(y_treino, dtype=torch.float32)
X_teste = torch.tensor(X_teste, dtype=torch.float32)
y_teste = torch.tensor(y_teste, dtype=torch.float32)

Vamos checar os dados para ver como estão.



In [5]:
print(X_treino)
print()
print(y_treino)

tensor([[0.0686, 0.5611, 0.2885, 0.4777, 0.0876, 0.1022],
        [0.0624, 0.5250, 0.2500, 0.4730, 0.0871, 0.0994],
        [0.0478, 0.4917, 0.2692, 0.4572, 0.0829, 0.0934],
        ...,
        [0.1185, 0.5333, 0.2692, 0.5410, 0.0995, 0.1142],
        [0.0166, 0.5139, 0.2885, 0.3929, 0.0708, 0.0811],
        [0.0333, 0.5361, 0.2692, 0.4209, 0.0776, 0.0890]])

tensor([[0.0676],
        [0.0586],
        [0.0434],
        ...,
        [0.1527],
        [0.0172],
        [0.0216]])


### Criando a rede neural



Para criar uma rede neural usando `pytorch` é necessário criar uma classe. Observe aqui que a classe criada é baseada na classe `nn.Module` do `pytorch`. Classes podem ser baseadas em outras classes! Trata-se da característica de herança das classes.

Observe que definimos as camadas da rede neural dentro de um objeto `nn.Sequential`.

Observe também o método `forward`. Este funciona de forma similar ao dunder `__call__` que vimos anteriormente. O `pytorch` requer que usemos o `forward` e não o `__call__`, então aceitamos e seguimos em frente.



In [6]:
class MLP(nn.Module):
    def __init__(
        self, num_dados_entrada, neuronios_c1, neuronios_c2, num_targets
    ):
        # Temos que inicializar a classe mãe
        super().__init__()

        # Definindo as camadas da rede
        self.camadas = nn.Sequential(
                    nn.Linear(num_dados_entrada, neuronios_c1),
                    nn.ReLU(),
                    nn.Linear(neuronios_c1, neuronios_c2),
                    nn.ReLU(),
                    nn.Linear(neuronios_c2, num_targets),
                )

    def forward(self, x):
        """Esse é o método que executa a rede do pytorch."""
        x = self.camadas(x)
        return x

Agora podemos criar a nossa rede neural!



In [7]:
NUM_DADOS_DE_ENTRADA = X_treino.shape[1]
NUM_DADOS_DE_SAIDA = y_treino.shape[1]
NEURONIOS_C1 = 50
NEURONIOS_C2 = 20

minha_MLP = MLP(NUM_DADOS_DE_ENTRADA, NEURONIOS_C1, NEURONIOS_C2, NUM_DADOS_DE_SAIDA)

E podemos checar os parâmetros internos dela!



In [8]:
for p in minha_MLP.parameters():
    print(p)

Parameter containing:
tensor([[-0.3424, -0.1547, -0.3895,  0.3338,  0.3774,  0.0580],
        [-0.0637, -0.2031, -0.1094, -0.3476, -0.2957, -0.3424],
        [ 0.1933, -0.3319,  0.3310,  0.1541, -0.3546, -0.2388],
        [ 0.3379, -0.1359, -0.1796,  0.0634,  0.2852,  0.2523],
        [-0.3119, -0.1381,  0.1713,  0.2795,  0.0254, -0.0039],
        [ 0.1557,  0.2409,  0.3984, -0.0897, -0.2252,  0.0844],
        [-0.1813,  0.0557, -0.0736,  0.3029,  0.1276, -0.1468],
        [ 0.3139,  0.2386,  0.3397,  0.1198, -0.1394,  0.0251],
        [ 0.0033, -0.1836, -0.0964,  0.0270,  0.0228,  0.0519],
        [ 0.0227, -0.0901, -0.3467,  0.1761, -0.0460,  0.1649],
        [-0.2169,  0.1379,  0.3297,  0.2931,  0.1460,  0.3354],
        [-0.1625,  0.2588,  0.0833,  0.2703, -0.0058,  0.0499],
        [ 0.3666,  0.1508, -0.2310,  0.0965, -0.0393, -0.4063],
        [-0.2086,  0.0356,  0.1476,  0.0676,  0.2871,  0.3214],
        [ 0.3486, -0.0134, -0.0078,  0.3608,  0.1384, -0.3679],
        [ 0.3017, 

Também podemos realizar uma previsão, mas ela provavelmente será bem ruim!



In [9]:
y_prev = minha_MLP(X_treino)
y_prev

tensor([[-0.0768],
        [-0.0779],
        [-0.0810],
        ...,
        [-0.0756],
        [-0.0823],
        [-0.0794]], grad_fn=<AddmmBackward>)

### A função de perda e o otimizador



O `pytorch` já vem com diversas funções de perda já implementadas. A que computa o erro quadrático médio, por exemplo, já está pronta!

Fora isso, precisamos definir o nosso otimizador. O otimizador é quem cuida de atualizar os parâmetros da nossa rede. Nós implementamos na nossa rede em Python puro a descida do gradiente. Aqui usaremos um otimizador que é uma modificação da descida do gradiente, chamado `Adam`. Trata-se, simplificadamente, de uma descida do gradiente com taxa de aprendizado individualizada para cada parâmetro e que se altera ao longo do aprendizado. `Adam` é um poderosíssimo otimizador! Não tem porque não usarmos ele neste caso.



In [10]:
TAXA_DE_APRENDIZADO = 0.001

# função perda será o erro quadrático médio
fn_perda = nn.MSELoss()

# otimizador será o Adam, um tipo de descida do gradiente
otimizador = optim.Adam(minha_MLP.parameters(), lr=TAXA_DE_APRENDIZADO)

### O treino da rede



Para avisar o `pytorch` que iremos treinar a rede, vamos invocar o método `train` da nossa rede. Toda rede recém-criada já está no modo treino, mas aqui deixarei explícito para ficar bem claro.



In [11]:
minha_MLP.train()

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=6, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)

Observe que o ciclo do treino tem as mesmas etapas que vimos no notebook anterior, nada de novo! Aqui basta observar como executamos essas etapas no `pytorch`.



In [12]:
NUM_EPOCAS = 1000

y_true = y_treino

for epoca in range(NUM_EPOCAS):
    # forward pass
    y_pred = minha_MLP(X_treino)

    # zero grad
    otimizador.zero_grad()

    # loss
    loss = fn_perda(y_pred, y_true)

    # backpropagation
    loss.backward()

    # atualiza parâmetros
    otimizador.step()

    # mostra resultado
    print(epoca, loss.data)

0 tensor(0.1212)
1 tensor(0.1139)
2 tensor(0.1073)
3 tensor(0.1013)
4 tensor(0.0956)
5 tensor(0.0902)
6 tensor(0.0850)
7 tensor(0.0801)
8 tensor(0.0754)
9 tensor(0.0710)
10 tensor(0.0668)
11 tensor(0.0629)
12 tensor(0.0593)
13 tensor(0.0560)
14 tensor(0.0530)
15 tensor(0.0502)
16 tensor(0.0478)
17 tensor(0.0457)
18 tensor(0.0438)
19 tensor(0.0423)
20 tensor(0.0411)
21 tensor(0.0401)
22 tensor(0.0394)
23 tensor(0.0389)
24 tensor(0.0386)
25 tensor(0.0385)
26 tensor(0.0385)
27 tensor(0.0387)
28 tensor(0.0388)
29 tensor(0.0389)
30 tensor(0.0390)
31 tensor(0.0391)
32 tensor(0.0390)
33 tensor(0.0389)
34 tensor(0.0387)
35 tensor(0.0384)
36 tensor(0.0381)
37 tensor(0.0377)
38 tensor(0.0373)
39 tensor(0.0369)
40 tensor(0.0365)
41 tensor(0.0361)
42 tensor(0.0357)
43 tensor(0.0354)
44 tensor(0.0350)
45 tensor(0.0347)
46 tensor(0.0345)
47 tensor(0.0342)
48 tensor(0.0340)
49 tensor(0.0338)
50 tensor(0.0336)
51 tensor(0.0334)
52 tensor(0.0332)
53 tensor(0.0330)
54 tensor(0.0328)
55 tensor(0.0326)
56

Após o treino, podemos checar a performance da nossa rede. Observe a linha que inicia com `with`. Tudo que estiver no bloco do `with torch.no_grad()` é computado normalmente, porém o grafo computacional e os gradientes locais não são computados. Use isso sempre que for realizar contas com tensores fora do treino!!



In [13]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_treino)
    y_pred = minha_MLP(X_treino)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

[1577.99996901] [1496.95133543]
[1409.99999118] [1510.83803988]
[1128.99997044] [1151.65485334]
[12991.99967575] [11874.52911377]
[14720.0001564] [14586.95789337]
[2938.99994564] [2731.24564171]
[6635.00014687] [7926.8757]
[11215.99973297] [9353.14842415]
[1040.99999189] [1451.42604113]
[5040.00003624] [5143.41898346]
[9541.99988747] [8481.63610649]
[3792.99988461] [5411.54138374]
[1002.0000155] [748.90597296]
[2542.00005341] [1545.07868147]
[6788.00001717] [6727.25405693]
[9386.00025749] [8146.45237541]
[1245.99996853] [1724.53426027]
[997.00000262] [576.27674913]
[708.00000262] [772.72407579]
[1435.99997544] [1428.28318739]
[2590.00006676] [2638.05671835]
[1392.99997497] [1680.72740602]
[11755.99946976] [9508.30468178]
[2007.99997377] [1326.29356337]
[2707.99998474] [2713.22503471]
[631.00000417] [777.12587309]
[652.99999881] [1046.19307089]
[2492.00006247] [2601.33364058]
[594.99999416] [580.26457739]
[760.0000056] [1123.51212454]
[1179.00003028] [1095.20430422]
[1789.9999361] [1823

### O teste da rede



Vamos supor que você já testou diversas arquiteturas de rede e está pronto para usar os dados de teste. Nesta etapa precisamos indicar para o `pytorch` que o treino acabou e estamos na etapa de avaliação. Fazemos isso rodando o método `eval`.



In [14]:
minha_MLP.eval()

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=6, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)

Agora podemos realizar o teste!



In [15]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_teste)
    y_pred = minha_MLP(X_teste)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

[523.00000858] [824.21333551]
[628.00000334] [627.61221743]
[442.00000328] [738.04851866]
[2284.99997044] [2540.19768953]
[1040.99999189] [840.14011908]
[809.99999654] [1241.73914671]
[3049.99987316] [3455.97644424]
[720.00000596] [687.15458059]
[877.99999249] [581.92100477]
[1292.00000429] [1630.08813334]
[7056.00004578] [4852.1876297]
[4176.99999142] [5854.08641434]
[13701.00020599] [14134.10001755]
[2948.0000515] [4070.33406067]
[16732.99985504] [14400.16028976]
[1815.00000048] [1997.78414392]
[16220.00016022] [12199.48874664]
[5967.00019073] [4704.32626724]
[473.00000042] [606.77430582]
[1078.9999795] [1008.38711405]
[452.00000319] [674.63628912]
[801.99998283] [773.50681353]
[2458.99994993] [2478.83959532]
[2492.99998236] [2496.65046167]
[582.00000203] [691.48278952]
[1405.9999671] [1509.19594431]
[8715.99991035] [11667.42993927]
[7634.00004578] [6411.43977737]
[506.99999839] [704.51482153]
[665.99999094] [732.72893381]
[1815.00000048] [1640.4481082]
[707.00001383] [679.15053606]


E, finalmente, computar alguma métrica para medir a performance do nosso modelo.



In [16]:
RMSE = mean_squared_error(y_true, y_pred, squared=False)
print(f'Loss do teste: {RMSE}')

Loss do teste: 1379.0764506388527


## Conclusão



Em sequêmcia ao notebook anterior (**experimento R.07**), esse possúi o objetivo de treinar uma rede neural artificial tipo Multilayer Perceptron usando pytorch. Assim, é ressaltado os motivos para utilizar PyTorch, como sua otimização eficiente, ampla variedade de funcionalidades prontas para uso, suporte ao treinamento em GPUs e sua relevância tanto na academia quanto no mundo corporativo, sendo bem mais eficiente do que a rede neural que foi criada no decorrer do semestre.

Desse modo, são apresentadas as etapas do processo de treinamento da rede neural, onde os dados utilizados são provenientes do conjunto de dados de diamantes, esse que é um velho conhecido, sendo bastante usado na disciplina de machine learning, portatnto, os dados foram divididos em conjuntos de treinamento e teste. Assim, antes de prosseguir, os dados são normalizados usando o `MinMaxScaler` do **scikit-learn** para garantir que estejam na escala adequada para a rede neural.

Dessa maneira, o próximo passo é a criação da rede neural utilizando a classe `nn.Module` do PyTorch. Dessa maneira, a arquitetura da rede é definida dentro de um objeto `nn.Sequential`, com camadas lineares e funções de ativação `ReLU`, onde a rede foi iniciada com parâmetros aleatórios. Assim, após a criação da rede neural, é definida a **função de perda (mean squared error) e o otimizador (Adam)**. Já o treinamento da rede é realizado por um número especificado de épocas, onde ocorre o cálculo do `forward pass`, do reset dos gradientes, da perda (loss) e do `backpropagation` para atualizar os parâmetros da rede utilizando o otimizador, em busca de melhores valores.

Após concluir o treinamento, este que não comprova muito os acertos, pois é enviezado, levando em conta que os dados já são conhecidos, então a performance da rede é avaliada utilizando os dados de teste, esses sim comprovarão o quão eficiente a mesma será. Nessa perspectiva, a rede é colocada no modo de avaliação (eval) e os resultados são obtidos através do `forward pass` nos dados de teste. Em seguida, é possível computar métricas para medir a performance do modelo, como o erro quadrático médio (RMSE), onde é possível visualizar o valor encontrado de 1379.0764506388527 que é um pouco superior ao método da floresta aleatória, mas certamente pode ser aperfeiçoado e ultrapassar os métodos testados anteriormente nesse conjunto de dados de diamantes.

Para concluir, é importante salientar que o objetivo de treinar uma rede neural artificial utilizando PyTorch foi alcançado. Assim, através do exemplo apresentado, é possível compreender as etapas envolvidas no treinamento de uma rede neural, desde a preparação dos dados até a avaliação do modelo, além de compreender como usar a biblioteca do PyTorch, esta que é especializada em redes neurais e é muito eficiente, completa e amplamente sendo utilizada em diversos campos e sendo muito útil para trabalhos futuros. 

## Playground

